# Quantum Computing Introduction - Assignment 5
# Names: Lilith Bertens, Quincy Koelman
# IDs: BERTENL, KOELMAQ (500878956)

Welcome to the fifth assignment! During this assignment, you will use the IBM Qiskit package. This framework offers a huge variety of very interesting functionalities to explore. This assignment will require you to investigate about the proper usage of the tool. Please refer to the [IBM Quantum Documentation](https://qiskit.org/documentation/).

For this assignment you can use whatever simulator you consider useful to solve the exercises. Read carefully the questions and identify the most suitable simulator for your purposes. Moreover, you will have to setup your own [QuantumCircuit](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#quantumcircuit) because we are not going to use only 2-qubit quantum system. Hence, your solution should include the complete setup process to create and simulate the requested quantum circuit.

## Question 1

In the assignent on Quantum Programming 2 (Assignment 4), we mentioned the concept of _total equivalence_, namely, the identity of the output states (modulo a global phase difference) between the two circuits under verification. Moreover, you showed the total equivaence of the following quantum circuits.

![Question1](img/qci_a5_question1.png)

Additionally, as shown during the workshop on Quantum Programming 2, different quantum processors deal with the quantum circuits in different ways, depending on the topology, number of qubits, set of available gates, etc.. Perform the following steps:

1. Implement the quantum circuit in the left,
2. Draw the transpiled circuit from it.
3. Show how the Swap gate is actually implemented in your preferred backend.

In [2]:
# Solution
import qiskit
qc1 = qiskit.QuantumCircuit(2)
qc2 = qiskit.QuantumCircuit(2)
qc1.x(0)
qc2.x(0)
qc1.h(0)
qc2.h(0)
print(qiskit.quantum_info.Statevector(qc1))
print(qiskit.quantum_info.Statevector(qc2))
qc1.swap(0,1)
qc2.cx(0,1)
qc2.cx(1,0)
qc2.cx(0,1)
qc1.t
print(qiskit.quantum_info.Statevector(qc1))
print(qiskit.quantum_info.Statevector(qc2))
print(qc1)
print(qc2)

## Question 2

Preparing for the upcoming assessment, one of your classmates identified a quantum circuit supposed to prepare one of the Bell states (specifically $\ket{\Phi^{+}}$). As usual, you are not completely sure about its validity, hence your are going to implement such circuit in Qiskit to figure it out. Assume $\ket{q_{1}q_{0}}$ = $\ket{00}$. 

Perform the following steps:

1. Implement the following circuit.
2. Execute the simulator.
3. Print the resulting vector state.
4. Does the proposed circuit prepare the corresponding Bell state?

![Question2](img/qci_a5_question2.png)

In [3]:
# Solution



## Question 3

Moreover, your classmate claims that by manipulating the initial state (using something different from $\ket{00}$), you can also prepare another Bell state ($\ket{\Psi^{-}}$).

1. Could you confirm that?
2. What should you do with the initial state to produce the requested Bell state?

In [4]:
# Solution



## Question 4

At this point, you have 2 different quantum circuits for the preparation of the Bell states. They are totally equivalent; however, their actual implementation on a quantum processor is different.

![Question4](img/qci_a5_question4.png)

Perform the following steps:

1. Draw the transpiled circuit that you obtained from these circuits, using your preferred backend.
2. Explain the actual quantum gates used for the tranpilation process.

In [5]:
# Solution



## Question 5

During the study week, another classmate learn the concept of _partial equivalence_ of quantum circuits. Two circuits are partially equivalent if, given any valid initial input state, they exhibit the same probability for each measurement outcome. Basically, two circuits exhibit the same probability for every possible measurement outcome. Moreover, not all qubits of a circuit need to be measured and there can be a set of initial states to be verified
not just a particular single initial state.

By applying this proposed definition, your classmate identified the following two circuits and she claims that they are partially equivalent.

![Question5](img/qci_a5_question5.png)

1. Can you verify that your classmate's claim holds for the initial state $\ket{q_{2}q_{1}q_{0}} = \ket{000}$?

In [6]:
# Solution



 ## Question 6

 The definition of _partial equivalence_ mentions `any valid input state`; however, your previous answer only consider one initial state ($\ket{q_{2}q_{1}q_{0}} = \ket{000}$). Perform the following steps:
 
 1. Could you verify the claim with a random initial quantum state?
 2. Show the probability distribution functions for each circuit.
 3. Explain whether they are the same or not.

In [7]:
# Solution



## Question 7

During the lecture: "Fundamentals: A Multi-Qubit World (2)", I introduced the Greenberger-Horne-Zeilinger (GHZ) state for a 3-qubit quantum system. Perform the following steps:

1. Assemble a circuit that creates a 5-qubit GHZ state.
2. Draw the circuit.
3. Show the resulting state vector.

In [8]:
# Solution



## Question 8

During the lecture: "Fundamentals: A Multi-Qubit World (2)", I presented the following quantum circuit for parity calculation on communication systems:

![Question8](img/qci_a5_question8.png)

Assuming $\ket{q_{2}q_{1}q_{0}}$ = $\ket{000}$, perform the following steps:

1. Assemble, draw and simulate the appropriate circuit.
2. Show the resulting probabilities of the simulations.

NB: The circuit presented during the lecture only measures the `parity qubit` (the most significative qubit). For completeness, in this exercise you should measure all 3 qubits.

In [9]:
# Solution



## Question 9

At this point you should be aware that n-qubits system $\rightarrow 2^{n}$ distinguishable states. In the previous exercise (`Question 8`) you had a 3-qubits system, hence you should have expected 8 distinguishable states yet there are only 4.

Considering the obtained results (4 distinguishable states if you previous solution is correct), explain how can you use them to back up the claim: `"State collapsed to the given parity measurement!"` presented during the lecture. The answer to this question does not require a python code. You just have to analyze the results and provide an answer.

### Write down your answer here

## Question 10

During the lecture: "Fundamentals: A Multi-Qubit World (2)", I introduced the Quantum Teleportation algorithm.

1. Implement, simulate and draw the teleportation quantum circuit.
2. Show the initial and the resulting vector state (you can use the Bloch sphere).
3. Explain how do you recognize that the algorithm works.

![Question10](img/qci_a5_question10.png)

NB: In order to better show the result of your algorithm, you should initialize your quantum circuit to a random or predefined vector state. You could use the `circuit.initialize()` command to perform such task.

In [10]:
# Solution

